In [ ]:
# Money Supply
leading_indicators = 'M2SL'
equities = ['BTC-USD','GLD','SLV']



import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import torch
import torch.nn as nn
import torch.optim as optim
import mlflow
import mlflow.pytorch
import logging
from pathlib import Path
from dash import Dash, html, dcc, dash_table
import plotly.express as px
from joblib import Parallel, delayed

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

def calculate_metrics(y_true, y_pred):
    """Calculate RMSE, MAE, and R²."""
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    return {'rmse': rmse, 'mae': mae, 'r2': r2}

def data_stock_data(ticker):
    """
    Fetch monthly adjusted stock data from yfinance.
    """
    output_dir = Path("C:/Users/Steel/Desktop/Projects/intel-sweep/intel-sweep/src/data/yf_monthly")
    logger.info(f"Fetching monthly data for {ticker}")
    # Use yfinance download with monthly interval
    try:
        #Maybe we already have the data saved locally
        data = pd.read_csv(f"C:/Users/Steel/Desktop/Projects/intel-sweep/intel-sweep/src/data/yf_monthly/{ticker}_monthly.csv")
    except FileNotFoundError:
        #If not, download it
        data = yf.download(ticker, period="max", interval="1mo", auto_adjust=True, progress=False)
        if data.empty:
            raise ValueError(f"No data returned for {ticker}")

        # Reset and rename columns for consistency with Alpha Vantage
        data.reset_index(inplace=True)
        data.rename(columns={
            'Date': 'Date',
            'Open': 'Open',
            'High': 'High',
            'Low': 'Low',
            'Close': 'Close',
            'Volume': 'Volume'
        }, inplace=True)
        # Add 'Adj Close' (already adjusted due to auto_adjust=True)
        data.columns = data.columns.get_level_values(0)
        data['Adj Close'] = data['Close']

        # Keep relevant columns
        data = data[['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']]
        data['Date'] = pd.to_datetime(data['Date']).dt.strftime('%Y-%m-%d')
        data = data.sort_values('Date')
        data = data.round(4)
        output_file = output_dir / f"{ticker.upper()}_monthly.csv"
        data.to_csv(output_file, index=True)
    return data


def get_indicator_data(series_id, start_date="2000-01-01", end_date="2025-04-30",API_KEY= "17188a6953269ab608ba14c3e3d8fb02" ):
    print("Test")
    try:
        #Try to read the CSV file first
        df = pd.read_csv(f"C:/Users/Steel/Desktop/Projects/intel-sweep/intel-sweep/src/data/fred_economic_indicators/{series_id}.csv", index_col="date", parse_dates=True)
        
        print("Ttest3")
        return df
    except Exception as e:
        print("Test2")
        base_url = "https://api.stlouisfed.org/fred/series/observations"
        long_series = []
        # If the CSV file does not exist or cannot be read, fetch from FRED API
        alt_start_date = "2017-06-01" if series_id in long_series else start_date
        params = {
            "series_id": series_id,
            "api_key": API_KEY,
            "realtime_start": alt_start_date,
            "realtime_end": end_date,
            "observation_start": alt_start_date,
            "observation_end": end_date,
            # "frequency": frequency,
            "file_type": "json"
        }

        try:
            response = requests.get(base_url, params=params)
            print(response.content)
            response.raise_for_status()
            data = response.json()
            
            if "observations" not in data or not data["observations"]:
                logger.warning(f"No observations for {series_id}")
                return None
                
            df = pd.DataFrame(data["observations"])[["date", "value"]]
            df["value"] = pd.to_numeric(df["value"], errors="coerce")
            df["date"] = pd.to_datetime(df["date"])
            df = df.rename(columns={"value": series_id}).set_index("date")
            df.to_csv(f"C:/Users/Steel/Desktop/Projects/intel-sweep/intel-sweep/src/data/fred_economic_indicators/{series_id}.csv", index=True)
            return df
        except requests.exceptions.RequestException as e:
            print(e)
            logger.error(f"Error fetching {series_id}: {e}")
            return None

def merge_indicators_with_stock_data(stock_list, indicators, start_date="2000-01-01", end_date="2025-04-30"):
    """Merge stock data with economic indicators."""
    logger.info("Merging stock data with economic indicators")
    merged_data =  get_stock_data(stock_list[0], start_date, end_date)
    for stock in stock_list[1:]:
        stock_data = get_stock_data(stock, start_date, end_date)
        if stock_data is not None:
            merged_data = merged_data.join(indicator_data, how='left')


    for indicator in indicators:
        indicator_data = get_indicator_data(indicator, start_date, end_date)
        if indicator_data is not None:
            merged_data = merged_data.join(indicator_data, how='left')
            logger.info(f"Added indicator {indicator} to stock data")
        else:
            logger.warning(f"Indicator {indicator} data not available")

    # Fill NaN values with forward fill and backward fill
    merged_data.fillna(method='ffill', inplace=True)
    merged_data.fillna(method='bfill', inplace=True)
    merged_data.to_csv(f"C:/Users/Steel/Desktop/Projects/intel-sweep/intel-sweep/data/combined/m2-bitcoin-gold_combined.csv")
    return merged_data

def engineer_features(df: pd.DataFrame, target_col: str, lag_periods: List[int] = [1, 3, 6], ma_windows: List[int] = [3, 12]) -> pd.DataFrame:
    """Add lagged variables and moving averages for features."""
    try:
        logger.info("Engineering features: lags and moving averages")
        df = df.copy()
        
        # Create lagged features for all columns
        for col in df.columns:
            for lag in lag_periods:
                df[f'{col}_lag{lag}'] = df[col].shift(lag)
        
        # Create moving averages for target column (stock price)
        for window in ma_windows:
            df[f'{target_col}_ma{window}'] = df[target_col].rolling(window=window).mean()
        
        # Drop rows with NaN values from lags or moving averages
        df = df.fillna(method='bfill').fillna(method='ffill')
        logger.info(f"Feature engineering complete, resulting shape: {df.shape}")
        return df
    except Exception as e:
        logger.error(f"Error in feature engineering: {str(e)}")
        raise


class StockPriceNN(nn.Module):
    """Feed-forward neural network for stock price forecasting."""
    def __init__(self, input_size, hidden_size=64):
        super(StockPriceNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, hidden_size // 2)
        self.fc3 = nn.Linear(hidden_size // 2, 1)

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

class StockPriceLSTM(nn.Module):
    """LSTM network for stock price forecasting."""
    def __init__(self, input_size, hidden_size=64, num_layers=2):
        super(StockPriceLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

def train_pytorch_forecast(data_path, target, selected_features, horizon=1, params=None, use_lstm=False):
    """Train PyTorch model for h-month-ahead forecasting."""
    print("Test: Entering train_pytorch_forecast")
    if params is None:
        params = {'hidden_size': 64, 'epochs': 100, 'learning_rate': 0.001, 'batch_size': 16}

    try:
        df = pd.read_parquet(data_path)
        logger.info(f"Loaded data for {target} with shape {df.shape}")
        if target not in df.columns:
            logger.error(f"Target column {target} not found in data.")
            return None, None, None

        df[target] = pd.to_numeric(df[target], errors='coerce')
        if df[target].isna().any():
            logger.warning(f"Target {target} contains {df[target].isna().sum()} NaN values after conversion.")
            df = df.dropna(subset=[target])
            if df.empty:
                logger.error(f"No valid data for {target} after numeric conversion.")
                return None, None, None
        logger.info(f"Target {target} dtype: {df[target].dtype}")

        if len(df) < horizon:
            logger.error(f"Dataset too short ({len(df)} rows) for horizon {horizon}.")
            return None, None, None

        df[target + f'_t{horizon}'] = df[target].shift(-horizon)
        df = df.dropna(subset=selected_features + [target + f'_t{horizon}'])
        logger.info(f"Data shape after NaN handling: {df.shape}")
        if df.empty:
            logger.error(f"No valid data for {target} after NaN handling.")
            return None, None, None

        X = df[selected_features].fillna(method="ffill").fillna(method="bfill")
        y = df[target + f'_t{horizon}']
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=False)
        logger.info(f"Train indices: {X_train.index[:5].tolist()}, Test indices: {X_test.index[:5].tolist()}")
        y_train = pd.to_numeric(y_train, errors='coerce')
        y_test = pd.to_numeric(y_test, errors='coerce')
        non_nan_train = ~y_train.isna()
        non_nan_test = ~y_test.isna()
        X_train = X_train[non_nan_train]
        y_train = y_train[non_nan_train]
        X_test = X_test[non_nan_test]
        y_test = y_test[non_nan_test]
        logger.info(f"Train size after NaN filter: {len(X_train)}, Test size after NaN filter: {len(X_test)}")
        if len(X_train) == 0 or len(X_test) == 0:
            logger.error(f"No valid train/test data for {target} after NaN handling.")
            return None, None, None

        print("Test2: Before scaling")
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)

        if use_lstm:
            # Reshape for LSTM [samples, timesteps, features]
            X_train_scaled = X_train_scaled.reshape(X_train_scaled.shape[0], 1, X_train_scaled.shape[1])
            X_test_scaled = X_test_scaled.reshape(X_test_scaled.shape[0], 1, X_test_scaled.shape[1])

        X_train_tensor = torch.FloatTensor(X_train_scaled)
        y_train_tensor = torch.FloatTensor(y_train.values).reshape(-1, 1)
        X_test_tensor = torch.FloatTensor(X_test_scaled)
        y_test_tensor = torch.FloatTensor(y_test.values).reshape(-1, 1)

        input_size = X.shape[1]
        model = StockPriceLSTM(input_size, params['hidden_size']) if use_lstm else StockPriceNN(input_size, params['hidden_size'])
        criterion = nn.MSELoss()
        optimizer = optim.Adam(model.parameters(), lr=params['learning_rate'])

        mlflow.set_experiment("stock_price_forecasting")
        with mlflow.start_run(run_name=f"{target}_horizon{horizon}"):
            mlflow.set_tag("ticker", target.split('_')[0])
            mlflow.set_tag("horizon", horizon)
            mlflow.set_tag("model_type", "LSTM" if use_lstm else "NN")
            model.train()
            for epoch in range(params['epochs']):
                for i in range(0, len(X_train_tensor), params['batch_size']):
                    batch_X = X_train_tensor[i:i + params['batch_size']]
                    batch_y = y_train_tensor[i:i + params['batch_size']]
                    optimizer.zero_grad()
                    outputs = model(batch_X)
                    loss = criterion(outputs, batch_y)
                    loss.backward()
                    optimizer.step()
                if (epoch + 1) % 10 == 0:
                    logger.info(f"Epoch {epoch+1}/{params['epochs']} - Loss: {loss.item():.4f}")

            model.eval()
            with torch.no_grad():
                y_pred = model(X_test_tensor).numpy().flatten()
            # Bias correction
            with torch.no_grad():
                y_train_pred = model(X_train_tensor).numpy().flatten()
            bias = np.mean(y_train_pred - y_train)
            y_pred += bias
            metrics = calculate_metrics(y_test, y_pred)
            logger.info(f"Metrics for {target} (horizon {horizon}): {metrics}")

            current_prices = df.loc[y_test.index, target]
            logger.info(f"Current prices shape: {current_prices.shape}, y_test shape: {y_test.shape}")
            price_change = (y_pred - current_prices) / current_prices * 100
            signals = np.where(price_change > 5, "Buy", np.where(price_change < -5, "Sell", "Hold"))
            backtest_df = pd.DataFrame({
                "Date": y_test.index,
                "Actual": y_test,
                "Predicted": y_pred,
                "Current_Price": current_prices,
                "Signal": signals
            })
            backtest_df.to_csv(f"backtest_{target}_horizon{horizon}.csv")
            mlflow.log_params(params)
            mlflow.log_metrics(metrics)
            mlflow.log_param("features", ", ".join(selected_features))
            mlflow.log_artifact(f"backtest_{target}_horizon{horizon}.csv")
            mlflow.pytorch.log_model(model, "pytorch_model", input_example=X_train_scaled[:1])
            model_uri = f"runs:/{mlflow.active_run().info.run_id}/pytorch_model"
            registered_model = mlflow.register_model(model_uri, f"StockPricePyTorch_Horizon{horizon}")
            logger.info(f"Model registered: {registered_model.name} version {registered_model.version}")

        return model, metrics, backtest_df
    except Exception as e:
        print(f"Error in train_pytorch_forecast for {target} horizon {horizon}: {str(e)}")
        logger.error(f"Error in train_pytorch_forecast for {target} horizon {horizon}: {str(e)}")
        return None, None, None




def create_dashboard(results):
    """Create a Dash dashboard for buy/sell signals."""


if __name__ == "__main__":
    print("Test: Main block")
    tickers = pd.read_csv("C:/Users/Steel/Desktop/Projects/intel-sweep/intel-sweep/src/data/equities.csv")['Symbol'].tolist()
    logger.info(f"Processing {len(tickers)} tickers: {tickers}")
    # Process a single ticker for testing
    results = process_stock("AAPL")
    print(f"Results: {results}")
    
    if results:
        # Create dashboard for AAPL
        create_dashboard(results)
        
        # Process all tickers
        results = Parallel(n_jobs=-1)(delayed(process_stock)(ticker) for ticker in tickers)  # Limit for demo
        results = [r for r in sum([r if r else [] for r in results], []) if r is not None]
        results_df = pd.DataFrame(results)
        if not results_df.empty:
            logger.info(f"Results DataFrame shape: {results_df.shape}")
            top_10 = results_df.sort_values(by='r2', ascending=False).head(10)
            logger.info(f"Top 10 models by R²:\n{top_10[['ticker', 'horizon', 'r2', 'rmse', 'mae']]}")
            top_10.to_csv("top_10_forecasts_by_r2.csv", index=False)

            with mlflow.start_run(run_name="Top_10_Forecasts_Summary"):
                mlflow.log_artifact("top_10_forecasts_by_r2.csv")
                for i, row in top_10.iterrows():
                    mlflow.log_metric(f"{row['ticker']}_horizon{row['horizon']}_r2", row['r2'])
                    mlflow.log_metric(f"{row['ticker']}_horizon{row['horizon']}_rmse", row['rmse'])
                    mlflow.log_metric(f"{row['ticker']}_horizon{row['horizon']}_mae", row['mae'])

            create_dashboard(results)  # Dashboard for all tickers
